## import modules

In [1]:
import cobra
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model
from cobra import Model, Reaction, Metabolite
from metnet import pathway
from cobra.flux_analysis import pfba
from cobra.flux_analysis.loopless import add_loopless, loopless_solution

## modify the strain

In [2]:
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    model = cobra.io.read_sbml_model('./iML1515.xml')

    ## add new metabolites
    ibutoh_e = Metabolite(
        'ibutoh_e',
        formula = 'C4H10O',
        name = 'isobutanol',
        compartment = 'e')
    
    ibutoh_c = Metabolite(
        'ibutoh_c',
        formula = 'C4H10O',
        name = 'isobutanol',
        compartment = 'c')
    
    mppal_c = Metabolite(
        'mppal_c',
        formula = 'C4H8O',
        name = '2_methylpropanal',
        compartment = 'c')
    
    model.add_metabolites(ibutoh_e)
    model.add_metabolites(ibutoh_c)
    model.add_metabolites(mppal_c)
    
    ## get the ids of relative metabolites in the metabolic network
    mob_c = model.metabolites.get_by_id('3mob_c')
    h_c = model.metabolites.get_by_id('h_c')
    co2_c = model.metabolites.get_by_id('co2_c')
    nadh_c = model.metabolites.get_by_id('nadh_c')
    nad_c = model.metabolites.get_by_id('nad_c')
    
    ## add new reactions
    model.add_reactions([Reaction('MOBDC')])
    model.reactions.MOBDC.add_metabolites({
        mob_c: -1.0,
        h_c: -1.0,
        co2_c: 1.0,
        mppal_c: 1.0
    })
    
    model.add_reactions([Reaction('ALCD23xi')])
    model.reactions.ALCD23xi.add_metabolites({
        h_c: -1.0,
        nadh_c: -1.0,
        mppal_c: -1.0,
        nad_c: 1.0,
        ibutoh_c: 1.0
    })
    
    model.add_reactions([Reaction('EX_ibutoh_e')])
    model.reactions.EX_ibutoh_e.add_metabolites({
        ibutoh_e: -1.0
    })
    
    model.add_reactions([Reaction('ibutohtex_reverse')])
    model.reactions.ibutohtex_reverse.add_metabolites({
        ibutoh_e: 1.0,
        ibutoh_c: -1.0
    })
    
    ## generate the metabolic network of the recombinant strain
    write_sbml_model(model,'../biosyntehsis-growth relationship/HSEC0916.xml')
    write_sbml_model(model,'../production prediction/HSEC0916.xml')
    write_sbml_model(model,'../metabolic engineering strategy optimization/HSEC0916.xml')